In [1]:
import pandas as pd
from tqdm import notebook
import re
import nltk

In [2]:
# формируем исходный датафрейм, путь прописывается вручную
data = pd.read_excel('/Users/artem/Downloads/СберБизнес. Выписка за 2022.09.01-2022.09.28 счёт 40702810538000113880.xlsx')
new_cols = ['column_0', 'column_1','column_2','column_3','column_4','column_5','column_6','column_7','column_8','column_9','column_10','column_11','column_12','column_13','column_15','column_16','column_17','column_18','column_19','column_20','column_21']
data.columns = new_cols
# column_4 - покупатели
# column_8 - поставщики
# column_9 - суммы по поставщикам

/Users/artem/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
# очищаем данные от букв и символов
for i in notebook.tqdm(range(len(data['column_8']))):
    text = str(data.loc[i, 'column_8'])
    text = re.sub(r'[^а-яА-ЯЁё]', ' ', text)
    text = text.split()
    text = " ".join(text)
    data.loc[i, 'column_8'] = text

  0%|          | 0/181 [00:00<?, ?it/s]

In [5]:
data = data.dropna(axis=0, how='all', subset='column_15').reset_index(drop=True)

In [6]:
data['column_9'] = data['column_9'].drop([0,1], axis=0)

In [7]:
# в расчетах банк не учитывает выдачу и погашение кредита как доход/расход, хотя в выписке они есть, поэтому столбец надо удалять
df_new = data.drop(data[data['column_8'] == "ПАО СБЕРБАНК"].index)

pivot_table_2 = df_new.pivot_table(index='column_8', values='column_9', aggfunc='sum').reset_index().drop(0, axis=0).sort_values(by='column_9', ascending=False)

x = pivot_table_2['column_9'].sum()

pivot_table_2['column_9'] = pivot_table_2['column_9'].astype('float64')
pivot_table_2['procent'] = (pivot_table_2['column_9']/x)*100
display(pivot_table_2)
# ниже вывод одного из 3 условий предоставления кредита
print('Сумма по трем крупнейшим контрагентам (не должна быть больше 80%): ', round(pivot_table_2['procent'].head(3).sum(), 2))

,column_8,column_9,procent
15,ООО ПЛАСТИК ПАЙП,6772432.97,31.015427
16,ООО ПОЛИМЕР Т,5214864.12,23.882294
17,ООО ПФ ТПК,1645863.04,7.537490
21,ООО ТД МИРТРУБПЛАСТ,1247500.80,5.713127
22,ООО ТД СВЯЗЬСТРОЙДЕТАЛЬ,1154961.60,5.289329
13,ООО Гринстройформ,1005741.34,4.605951
24,ООО Торговая Компания ПЛАСТИК ПАЙП,961490.00,4.403295
4,ИП Яшанин Виктор Алексеевич,582000.00,2.665361
23,ООО ТК ПЛАСТИК ПАЙП,580800.00,2.659865
34,УФК по г Москве ИФНС России по г Москве,516250.01,2.364249


Сумма по трем крупнейшим контрагентам (не должна быть больше 80%):  62.44


In [8]:
display(pivot_table_2[['column_8', 'procent']].head(10))

,column_8,procent
15,ООО ПЛАСТИК ПАЙП,31.015427
16,ООО ПОЛИМЕР Т,23.882294
17,ООО ПФ ТПК,7.537490
21,ООО ТД МИРТРУБПЛАСТ,5.713127
22,ООО ТД СВЯЗЬСТРОЙДЕТАЛЬ,5.289329
13,ООО Гринстройформ,4.605951
24,ООО Торговая Компания ПЛАСТИК ПАЙП,4.403295
4,ИП Яшанин Виктор Алексеевич,2.665361
23,ООО ТК ПЛАСТИК ПАЙП,2.659865
34,УФК по г Москве ИФНС России по г Москве,2.364249
